# SHA-1 algorithm

In [214]:
import numpy as np
import binascii

## Help functions

In [215]:
LENGTH_BITS = 64
BLOCK_SIZE = 512

In [216]:
def str_to_bits(string: str) -> np.ndarray:
    bytes_array = bytes(string, 'utf-8')
    array = np.frombuffer(bytes_array, dtype=np.uint8)
    bits_array = np.unpackbits(array)
    return bits_array

def int_to_x_bits(number: int, bits) -> np.ndarray:
    binary_string = np.binary_repr(number, width=bits)
    return np.array(list(binary_string), dtype=np.uint8)

def int_to_64_bits(number: int) -> np.ndarray:
    return int_to_x_bits(number, 64)

def int_to_32_bits(number: int) -> np.ndarray:
    return int_to_x_bits(number, 32)

def bits_to_int(bits_array):
    binary_string = ''.join(bits_array.astype(str))
    return int(binary_string, 2)

def binary_negation(bits: np.ndarray) -> np.ndarray:
    return np.vectorize(lambda x: (x + 1) % 2)(bits)

def add_padding(bits: np.ndarray) -> np.ndarray:
    bits_len = len(bits)
    padding_len = BLOCK_SIZE - (bits_len + LENGTH_BITS + 1) % BLOCK_SIZE
    one_padding = np.ones(1)
    padding = np.zeros(padding_len)
    bits_with_one_padding = np.append(bits, one_padding)
    bits_with_padding = np.append(bits_with_one_padding, padding)
    return bits_with_padding

def add_length_bits(bits: np.ndarray, bits_len: int) -> np.ndarray:
    length_bits = int_to_64_bits(bits_len)
    bits_with_length = np.append(bits, length_bits)
    return bits_with_length

def split_x_blocks(bits: np.ndarray, size: int) -> np.ndarray:
    assert bits.size % size == 0
    num_blocks = bits.size // size
    blocks = np.array_split(bits, num_blocks)
    return np.array(blocks, dtype=np.uint8)

def split_512_blocks(bits: np.ndarray) -> np.ndarray:
    return split_x_blocks(bits, 512)

def split_32_blocks(bits: np.ndarray) -> np.ndarray:
    return split_x_blocks(bits, 32)

def left_rotate(arr, rotations):
    rotations = rotations % arr.size
    return np.roll(arr, -rotations)

def add_modulo(a_bits, b_bits):
    a = bits_to_int(a_bits)
    b = bits_to_int(b_bits)
    res = (a + b) & 0xFFFFFFFF
    return int_to_32_bits(res)


In [217]:
def extend_chunks(chunks: np.ndarray) -> np.ndarray:
   for i in range(16, 80):
      new_chunk = chunks[i-3] ^ chunks[i-8] ^ chunks[i-14] ^ chunks[i-16]
      new_chunk = left_rotate(new_chunk, 1)
      chunks = np.append(chunks, [new_chunk], axis=0)
   return chunks

def process_block(block, var):
   chunks =  split_32_blocks(block)
   chunks = extend_chunks(chunks)

   a, b, c, d, e = var
   h0, h1, h2, h3, h4 = a, b, c, d, e

   for i, chunk in enumerate(chunks):
      if i < 20:
         f = (b & c) | (binary_negation(b) & d)
         k = int_to_32_bits(0x5A827999)
      elif i < 40:
         f = b ^ c ^ d
         k = int_to_32_bits(0x6ED9EBA1)
      elif i < 60:
         f = (b & c) | (b & d) | (c & d)
         k = int_to_32_bits(0x8F1BBCDC)
      elif i < 80:
         f = b ^ c ^ d
         k = int_to_32_bits(0xCA62C1D6)

      tmp = add_modulo(left_rotate(a, 5), f)
      tmp = add_modulo(tmp, e)
      tmp = add_modulo(tmp, k)
      tmp = add_modulo(tmp, chunk)
      e = d
      tmp_c = c
      c = left_rotate(b, 30)
      b = a
      a = tmp
      d = tmp_c
   
   h0 = add_modulo(h0, a)
   h1 = add_modulo(h1, b)
   h2 = add_modulo(h2, c)
   h3 = add_modulo(h3, d)
   h4 = add_modulo(h4, e)

   return (h0, h1, h2, h3, h4)


In [218]:
def sha1(data):
    bits = str_to_bits(data)
    bits_len = len(bits)
    bits = add_padding(bits)
    bits = add_length_bits(bits, bits_len)
    blocks = split_512_blocks(bits)

    h0 = int_to_32_bits(0x67452301)
    h1 = int_to_32_bits(0xEFCDAB89)
    h2 = int_to_32_bits(0x98BADCFE)
    h3 = int_to_32_bits(0x10325476)
    h4 = int_to_32_bits(0xC3D2E1F0)
    var = (h0, h1, h2, h3, h4)

    for block in blocks:
        var = process_block(block, var)
    
    h0, h1, h2, h3, h4 = var
    bits = h0
    bits = np.append(bits, h1)
    bits = np.append(bits, h2)
    bits = np.append(bits, h3)
    bits = np.append(bits, h4)
    hash_bytes = np.packbits(bits)
    return binascii.hexlify(hash_bytes).decode()

In [219]:
assert sha1("") == "da39a3ee5e6b4b0d3255bfef95601890afd80709"
assert sha1("abc") == "a9993e364706816aba3e25717850c26c9cd0d89d"
assert sha1("password") == "5baa61e4c9b93f3f0682250b6cf8331b7ee68fd8"
assert sha1("verrryyylooooooooooooong") == "0ae6369421330d5e25c5e9c0c4c6dc272870daf0"
assert sha1("DiFr33333N!") == "2489e5a1b1fae0ca5774913c5d0dc73e01cd0ae6"